เพื่อรันโน้ตบุ๊กต่อไปนี้ หากคุณยังไม่ได้ทำ คุณต้องปรับใช้โมเดลที่ใช้ `text-embedding-ada-002` เป็นโมเดลฐาน และตั้งชื่อการปรับใช้ในไฟล์ .env เป็น `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

ต่อไป เราจะโหลดดัชนีฝังตัว (Embedding Index) ลงใน Pandas Dataframe ดัชนีฝังตัวถูกเก็บไว้ในไฟล์ JSON ที่ชื่อว่า `embedding_index_3m.json` ดัชนีฝังตัวนี้ประกอบด้วยการฝังตัวสำหรับแต่ละคำบรรยายของ YouTube จนถึงปลายเดือนตุลาคม 2023


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

ต่อไป เราจะสร้างฟังก์ชันชื่อ `get_videos` ที่จะค้นหาใน Embedding Index สำหรับคำค้นหา ฟังก์ชันนี้จะคืนค่าวิดีโอ 5 อันดับแรกที่มีความคล้ายคลึงกับคำค้นหามากที่สุด ฟังก์ชันทำงานดังนี้:

1. ก่อนอื่น จะสร้างสำเนาของ Embedding Index
2. ต่อไป จะคำนวณ Embedding สำหรับคำค้นหาโดยใช้ OpenAI Embedding API
3. จากนั้น จะสร้างคอลัมน์ใหม่ใน Embedding Index ชื่อ `similarity` คอลัมน์ `similarity` จะเก็บค่าความคล้ายคลึงแบบโคไซน์ระหว่าง Embedding ของคำค้นหาและ Embedding ของแต่ละส่วนของวิดีโอ
4. ต่อไป จะกรอง Embedding Index โดยใช้คอลัมน์ `similarity` โดยกรองให้เหลือเฉพาะวิดีโอที่มีความคล้ายคลึงแบบโคไซน์มากกว่าหรือเท่ากับ 0.75
5. สุดท้าย จะเรียงลำดับ Embedding Index ตามคอลัมน์ `similarity` และคืนค่าวิดีโอ 5 อันดับแรก


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ฟังก์ชันนี้ง่ายมาก มันเพียงแค่พิมพ์ผลลัพธ์ของการค้นหาออกมาเท่านั้น


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ขั้นแรก จะโหลด Embedding Index ลงใน Pandas Dataframe  
2. ต่อมา ผู้ใช้จะถูกขอให้ป้อนคำค้นหา  
3. จากนั้นจะเรียกใช้ฟังก์ชัน `get_videos` เพื่อค้นหา Embedding Index สำหรับคำค้นหา  
4. สุดท้ายจะเรียกใช้ฟังก์ชัน `display_results` เพื่อแสดงผลลัพธ์ให้ผู้ใช้ดู  
5. จากนั้นผู้ใช้จะถูกขอให้ป้อนคำค้นหาอีกครั้ง กระบวนการนี้จะดำเนินต่อไปจนกว่าผู้ใช้จะป้อน `exit`  

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc.th.png)

คุณจะถูกขอให้ป้อนคำค้นหา ป้อนคำค้นหาแล้วกด enter แอปพลิเคชันจะแสดงรายการวิดีโอที่เกี่ยวข้องกับคำค้นหา แอปพลิเคชันยังจะแสดงลิงก์ไปยังตำแหน่งในวิดีโอที่คำตอบของคำถามนั้นอยู่ด้วย  

นี่คือตัวอย่างคำค้นหาที่คุณสามารถลองได้:  

- Azure Machine Learning คืออะไร?  
- เครือข่ายประสาทเทียมแบบ convolutional ทำงานอย่างไร?  
- เครือข่ายประสาทเทียมคืออะไร?  
- ฉันสามารถใช้ Jupyter Notebooks กับ Azure Machine Learning ได้หรือไม่?  
- ONNX คืออะไร?


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ข้อจำกัดความรับผิดชอบ**:  
เอกสารนี้ได้รับการแปลโดยใช้บริการแปลภาษาอัตโนมัติ [Co-op Translator](https://github.com/Azure/co-op-translator) แม้เราจะพยายามให้ความถูกต้องสูงสุด แต่โปรดทราบว่าการแปลอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่สำคัญ ขอแนะนำให้ใช้บริการแปลโดยผู้เชี่ยวชาญมนุษย์ เราไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความผิดใด ๆ ที่เกิดจากการใช้การแปลนี้
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
